# Signature Recognition and Detection Project
This project builds a deep learning model for signature recognition and detection. It uses CNN to classify signatures as genuine or forged.

In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


In [ ]:

# Load the dataset (assuming dataset is downloaded and placed in 'signature_data' folder)
DATASET_DIR = "signature_data"  # Define the directory where your signature dataset is stored

IMG_WIDTH, IMG_HEIGHT = 128, 128

def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT), color_mode='grayscale')
                img_array = image.img_to_array(img)
                images.append(img_array)
                labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_images_from_folder(DATASET_DIR)
images = images / 255.0

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], IMG_WIDTH, IMG_HEIGHT, 1))
X_test = np.reshape(X_test, (X_test.shape[0], IMG_WIDTH, IMG_HEIGHT, 1))

y_train_cat = to_categorical(y_train, num_classes=2)
y_test_cat = to_categorical(y_test, num_classes=2)


In [ ]:

model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1
)

datagen.fit(X_train)

history = model.fit(datagen.flow(X_train, y_train_cat, batch_size=32), 
                    epochs=20, 
                    validation_data=(X_test, y_test_cat))


In [ ]:

test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {test_acc}")


In [ ]:

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


In [ ]:

def predict_signature(model, img_path):
    img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT), color_mode='grayscale')
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    if prediction[0][0] > prediction[0][1]:
        print("Predicted: Genuine Signature")
    else:
        print("Predicted: Forged Signature")

# Example prediction (replace with actual image path)
example_image_path = 'path_to_signature_image.png'
predict_signature(model, example_image_path)
